In [0]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

import numpy as np
import argparse
import json
from logging.config import dictConfig
from logging import getLogger
import os
import time

In [0]:
class CNN(nn.Module):
    def __init__(self, n_class=10):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.AvgPool2d(8) #???
        )
        self.pc = nn.Sequential(
            nn.Linear(512, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, n_class),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 512)
        x = self.pc(x)
        return x

In [0]:
def train(model, device, train_loader, optimizer, criterion, logger):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            logger.debug("[train] batch : %s/%s (%.0f%%),\tloss : %.6f",
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
    return (None, loss.item())

In [0]:
def test(model, device, test_loader, criterion, logger):
    model.eval()
    test_loss = []
    correct = 0
    result = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss.append(criterion(output, target).item())
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            result += torch.eq(torch.max(output, 1).indices, target)
            
    test_loss = np.mean(np.array(test_loss))
    accuracy = 100. * correct / len(test_loader.dataset)
    
    logger.info("[test] ave loss : %.4f,\taccu : %d/%d(%.0f%%)",
        test_loss, correct, len(test_loader.dataset), accuracy)
    
    return (torch.tensor(result).numpy(), (test_loss, accuracy))

In [0]:
def main(description, option="", no_swap=False, random_swap=False):
    with open('logging.json') as f:
        dictConfig(json.load(f))
    logger = getLogger('env')
    logger.debug("<" * 40)
    logger.info("[system] start")
    logger.info("[meta] %s", description)
    start_time = time.time()

    def fetch_args(args=[]):
        parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
        parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                            help='input batch size for training (default: 64)')
        parser.add_argument('--epochs', type=int, default=14, metavar='N',
                            help='number of epochs to train (default: 14)')
        parser.add_argument('--trials', type=int, default=10, metavar='T')
        parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                            help='learning rate (default: 0.001)')
        parser.add_argument('--b1', type=float, default=0.9, metavar='B',
                            help='learning rate beta (default: 0.9)')
        parser.add_argument('--b2', type=float, default=0.999, metavar='B',
                            help='learning rate beta (default: 0.999)')
        parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                            help='learning rate momentum (default: 0.9)')
        parser.add_argument('--dataset', default="cifar10", metavar='D')
        parser.add_argument('--n-class', type=int, default=10, metavar='C')
        parser.add_argument('--data-size', type=int, default=25000, metavar='D')
        parser.add_argument('--model', default="cnn", metavar='M')
        parser.add_argument('--optimizer', default="adam", metavar='O')
        parser.add_argument('--criterion', default="cross_entropy_loss", metavar='C')
        parser.add_argument('--scheduler', default="", metavar='S')
        parser.add_argument('--t-max', type=int, default=10, metavar='T')
        parser.add_argument('--save-model', action='store_true', default=False,
                            help='For Saving the current Model')
        return parser.parse_args(args=args)
  
    args = fetch_args(option.split(" "))
    for arg in vars(args):
        logger.info("[param] %s=%s", arg, vars(args)[arg])
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    logger.info("[device] %s", device)
    
    #画像の変形処理
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    #CIFAR-10のtrain, testsetのロード
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    def subset_dataloader(set):
      return torch.utils.data.DataLoader(
          torch.utils.data.Subset(trainset, set), 
          batch_size=args.batch_size, **kwargs)

    # init
    def make_dataset(n_class, data_size):
      full_labels = []
      full_indices = np.arange(len(trainset))
      for (_, target) in subset_dataloader(full_indices):
        full_labels += target.numpy().tolist()

      pack = np.array([(i, full_labels[i]) for i in range(len(full_labels))
              if full_labels[i] < n_class])
      np.random.shuffle(pack)
      pack = pack[:data_size].tolist()

      indices = [i for (i, _) in pack]
      labels = [label for (_, label) in pack]
      return (np.array(indices), np.array(labels), np.array(full_labels))

    N = int(args.data_size)
    indices, _, labels = make_dataset(args.n_class, 2 * N)
    A, B = indices[:N], indices[N:]
    logger.debug("[debug] labels : %s", labels.tolist())

    # labels = []
    # full_indices = np.arange(len(trainset))
    # for (_, target) in subset_dataloader(full_indices):
    #   labels += target.numpy().tolist()
    # labels = np.array(labels)

    # indices = full_indices[lebels < args.n_class]
    # labels = labels[lebels < args.n_class]
    # logger.debug("[debug] labels : %s", labels.tolist())

    # N = int(args.data_size) #int(len(trainset) / 2)
    # indices = np.arange(len(trainset))[:2 * N]
    # np.random.shuffle(indices)
    # A, B = indices[:N], indices[N:]
    
    log = {"X_train":[], "X_test":[], "Y_train":[], "Y_test":[], "X_acc":[], "Y_acc":[],
          "atob": [], "btoa": []}


    for trial in range(args.trials):
      logger.debug("-" * 20)
      logger.info("[system] trial %d / %d", trial, args.trials)

      # log
      log["X_train"].append([])
      log["X_test"].append([])
      log["X_acc"].append([])
      if not no_swap: 
        log["Y_train"].append([])
        log["Y_test"].append([])
        log["Y_acc"].append([])

      # instantiate
      if True:
        model_X = CNN(n_class=args.n_class).to(device)
        if not no_swap: model_Y = CNN(n_class=args.n_class).to(device)

        if args.optimizer == "adam":
          optimizer_X = optim.Adam(model_X.parameters(), lr=args.lr, betas=(args.b1, args.b2))
          if not no_swap: optimizer_Y = optim.Adam(model_Y.parameters(), lr=args.lr, betas=(args.b1, args.b2))
        if args.optimizer == "adagrad":
          optimizer_X = optim.Adagrad(model_X.parameters(), lr=args.lr)
          if not no_swap: optimizer_Y = optim.Adagrad(model_Y.parameters(), lr=args.lr)
        if args.optimizer == "sdg":
          optimizer_X = optim.SGD(model_X.parameters(), lr=args.lr, momentum=args.momentum)
          if not no_swap: optimizer_Y = optim.SGD(model_Y.parameters(), lr=args.lr, momentum=args.momentum)
        if args.optimizer == "asdg":
          optimizer_X = optim.ASGD(model_X.parameters(), lr=args.lr)
          if not no_swap: optimizer_Y = optim.ASGD(model_Y.parameters(), lr=args.lr)

        scheduler_X = None
        if args.scheduler == "cosine_annealing":
          scheduler_X = optim.lr_scheduler.CosineAnnealingLR(optimizer_X, T_max=args.t_max, eta_min=1e-6)

        criterion = nn.CrossEntropyLoss()

      # Split
      A_loader, B_loader = subset_dataloader(A), subset_dataloader(B)
      
      def fit_X():
        logger.info("[system] X : train_a -> test_b")
        for epoch in range(args.epochs):
          logger.debug("-" * 20)
          logger.info("[system] epoch %d", epoch)

          (_, loss_XA) = train(model_X, device, A_loader, optimizer_X, criterion, logger)
          (testB, (loss_XB, acc_X)) = test(model_X, device, B_loader, criterion, logger)
          # if scheduler_X: scheduler_X.step() # うごいてなさそう
        
          log["X_train"][trial].append(loss_XA)
          log["X_test"][trial].append(loss_XB)
          log["X_acc"][trial].append(acc_X)
        return testB

      def fit_Y():
        logger.info("[system] Y : train_b -> test_a")
        for epoch in range(args.epochs):
          logger.debug("-" * 20)
          logger.info("[system] epoch %d", epoch)

          (_, loss_YB) = train(model_Y, device, B_loader, optimizer_Y, criterion, logger)
          (testA, (loss_YA, acc_Y)) = test(model_Y, device, A_loader, criterion, logger)
        
          log["Y_train"][trial].append(loss_YB)
          log["Y_test"][trial].append(loss_YA)
          log["Y_acc"][trial].append(acc_Y)
        return testA

      # training
      testB = fit_X()
      if not no_swap: testA = fit_Y()

      # swap
      if not no_swap:
        if random_swap:
          logger.info("[debug] random swap")
          testA = np.random.rand(len(testA)) < 0.5
          testB = np.random.rand(len(testB)) < 0.5
        # result
        A_o, A_x = A[testA], A[np.logical_not(testA)]
        B_o, B_x = B[testB], B[np.logical_not(testB)]

        # label
        A_label = [(i, labels[i]) for i in A_x]
        B_label = [(i, labels[i]) for i in B_o]
        logger.debug("[debug] A_x labels %s", A_label)
        logger.debug("[debug] B_o labels %s", B_label)
        
        # class batch
        A_trans, A_left = [], []
        B_trans, B_left = [], []
        for class_idx in range(args.n_class):
          A_class = [i for (i, label) in A_label if label==class_idx]
          B_class = [i for (i, label) in B_label if label==class_idx]
          s = min(len(A_class), len(B_class))
          A_trans += B_class[:s]
          A_left += A_class[s:]
          B_trans += A_class[:s]
          B_left += B_class[s:]
        logger.debug("[debug] B to A %s", A_trans)
        logger.debug("[debug] A to B %s", B_trans)
        
        # concat
        A = np.concatenate([A_o, A_trans, A_left]).astype(np.int64)
        B = np.concatenate([B_x, B_trans, B_left]).astype(np.int64)
        np.random.shuffle(A)
        np.random.shuffle(B)
        
      # log
      if not no_swap: 
        log["atob"].append(np.copy(A_trans).tolist())
        log["btoa"].append(np.copy(B_trans).tolist())

    # Statistics
    logger.debug("%s statistics %s", "-" * 10, "-" * 10)
    logger.info("[stat] X train loss : %s", log["X_train"])
    logger.info("[stat] X test loss : %s", log["X_test"])
    logger.info("[stat] X accuracy : %s", log["X_acc"])
    if not no_swap:
      logger.info("[stat] Y train loss : %s", log["Y_train"])
      logger.info("[stat] Y test loss : %s", log["Y_test"])
      logger.info("[stat] Y accuracy : %s", log["Y_acc"])
      logger.info("[stat] swap log A to B : %s", log["atob"])
      logger.info("[stat] swap log B to A : %s", log["btoa"])
    logger.info("[stat] elapsed time : %s[s]", time.time() - start_time)
    
    if args.save_model:
        logger.info("[system] saving...")
        torch.save(model.state_dict(), "swap.pt")
    
    logger.info("[system] finish")


In [0]:
def download_log():
  from google.colab import files
  files.download("./log.txt")

In [0]:
if __name__ == '__main__':
    main("デバッグ",
         option="--epochs 3 --batch-size 16 --optimizer sdg --data-size 100 --trials 2",
         no_swap=False)

In [0]:
if __name__ == '__main__':
    main("デバッグ class",
         option="--epochs 3 --batch-size 16 --optimizer sdg --data-size 100 --trials 2 --n-class=10")

In [0]:
if __name__ == '__main__':
    main("データスワップ : main : 2class",
         option="--epochs 20 --batch-size 16 --optimizer sdg --data-size 5000 --trials 15 --n-class=2")

In [0]:
if __name__ == '__main__':
    main("データスワップ : main : 2class",
         option="--epochs 20 --batch-size 16 --optimizer sdg --data-size 5000 --trials 15 --n-class=4")

[system] start
[meta] データスワップ : main : 2class
[param] batch_size=16
[param] epochs=20
[param] trials=15
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] n_class=4
[param] data_size=5000
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] scheduler=
[param] t_max=10
[param] save_model=False
[device] cuda


Files already downloaded and verified


[system] trial 0 / 15
[system] X : train_a -> test_b
[system] epoch 0
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
[test] ave loss : 1.3838,	accu : 1356/5000(27%)
[system] epoch 1
[test] ave loss : 1.3343,	accu : 1734/5000(35%)
[system] epoch 2
[test] ave loss : 1.2320,	accu : 2398/5000(48%)
[system] epoch 3
[test] ave loss : 1.1398,	accu : 2850/5000(57%)
[system] epoch 4
[test] ave loss : 1.1251,	accu : 2969/5000(59%)
[system] epoch 5
[test] ave loss : 1.0736,	accu : 3304/5000(66%)
[system] epoch 6
[test] ave loss : 1.1017,	accu : 3163/5000(63%)
[system] epoch 7
[test] ave loss : 1.0303,	accu : 3535/5000(71%)
[system] epoch 8
[test] ave loss : 1.0386,	accu : 3475/5000(70%)
[system] epoch 9
[test] ave loss : 1.0268,	accu : 3546/5000(71%)
[system] epoch 10
[test] ave loss : 1.0787,	accu : 3273/5000(65%)
[system] epo

In [0]:
if __name__ == '__main__':
    main("データスワップ : main",
         option="--epochs 30 --batch-size 16 --optimizer sdg --data-size 5000 --trials 10")

In [0]:
if __name__ == '__main__':
    main("データスワップ : random",
         option="--epochs 30 --batch-size 16 --optimizer sdg --data-size 5000 --trials 10",
         random_swap=True)

[system] start
[meta] データスワップ : random
[param] batch_size=16
[param] epochs=30
[param] trials=10
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] data_size=5000
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] scheduler=
[param] t_max=10
[param] save_model=False
[device] cuda


Extracting ./data/cifar-10-python.tar.gz to ./data


[system] trial 0 / 10
[system] X : train_a -> test_b
[system] epoch 0
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
[test] ave loss : 2.3025,	accu : 509/5000(10%)
[system] epoch 1
[test] ave loss : 2.3024,	accu : 539/5000(11%)
[system] epoch 2
[test] ave loss : 2.3022,	accu : 615/5000(12%)
[system] epoch 3
[test] ave loss : 2.3017,	accu : 696/5000(14%)
[system] epoch 4
[test] ave loss : 2.2987,	accu : 740/5000(15%)
[system] epoch 5
[test] ave loss : 2.2785,	accu : 826/5000(17%)
[system] epoch 6
[test] ave loss : 2.2633,	accu : 876/5000(18%)
[system] epoch 7
[test] ave loss : 2.2476,	accu : 925/5000(18%)
[system] epoch 8
[test] ave loss : 2.2490,	accu : 861/5000(17%)
[system] epoch 9
[test] ave loss : 2.2604,	accu : 822/5000(16%)
[system] epoch 10
[test] ave loss : 2.2530,	accu : 848/5000(17%)
[system] epoch 11
[test